<a href="https://colab.research.google.com/github/Krankile/ensemble_forecasting/blob/main/dataproc/notebooks/mase_divisor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
%%capture
!pip install kora

In [31]:
from kora import drive
drive.link_nbs()
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pathlib import Path 
from tqdm.notebook import tqdm


In [ ]:
WRITE_TO_FILE = False

#Clean data

In [32]:
ROOT = Path("/content/drive/MyDrive/Master, Ankile og Krange/Data")

In [7]:
hourly = pd.read_csv(ROOT / 'train/Hourly-train.csv').set_index("V1")
daily = pd.read_csv(ROOT / 'train/Daily-train.csv').set_index("V1")
weekly = pd.read_csv(ROOT / 'train/Weekly-train.csv').set_index("V1")
monthly = pd.read_csv(ROOT / 'train/Monthly-train.csv').set_index("V1")
quarterly = pd.read_csv(ROOT / 'train/Quarterly-train.csv').set_index("V1")
yearly = pd.read_csv(ROOT / 'train/Yearly-train.csv').set_index("V1")

In [8]:
all_data_map = {"Hourly": hourly, "Daily": daily, "Weekly": weekly, "Monthly":monthly, "Quarterly": quarterly,"Yearly": yearly} 

m_dict = {"Hourly":24, "Daily":1 , "Weekly":1 ,"Monthly":12 , "Quarterly":4 , "Yearly":1 } #As defined in M4 competition
forecast_horizons = {"Hourly":48, "Daily":14 , "Weekly":13 ,"Monthly":18 , "Quarterly":8 , "Yearly":6 } #As defined in M4 competition

100000

In [9]:
weights_train_path = ROOT / 'LPsolutions/optimal_weights_train_abs.feather'
weights_val_path = ROOT / 'LPsolutions/optimal_weights_val_abs.feather'

time_series_train = ROOT / 'Series/m4_series_n4_f32_train.feather'
time_series_val = ROOT / 'Series/m4_series_n4_f32_val.feather'

In [10]:
abs_weights_val = pd.read_feather(weights_val_path).set_index("idx")
time_series_val = pd.read_feather(time_series_val).set_index("st")

abs_weights_train = pd.read_feather(weights_train_path).set_index("idx")
time_series_train = pd.read_feather(time_series_train).set_index("st")

#Define a function to get all mase divisors

In [13]:
def calculate_mase_divisor(set_type):
    if "v" in set_type:
        weights = abs_weights_val
        time_series_meta = time_series_val
    else:
        weights = abs_weights_train
        time_series_meta = time_series_train

    mase_divisor = []
    
    for idx in tqdm(weights.index, desc=f"Processing the {set_type} set"):
        period = time_series_meta.loc[idx, "period"]
        
        h_slice = -forecast_horizons[period] if set_type != "test" else None
        h = forecast_horizons[period]
        m = m_dict[period]
        
        value_series = all_data_map[period].loc[idx].dropna().iloc[:h_slice]
        n = len(value_series)
        summ = 0
        for t in range(m, n): 
            summ += abs(value_series.iloc[t]-value_series.iloc[t-m])
        
        divisor = h*(1/(n-m))*summ

        mase_divisor.append(divisor)
    
    idx_and_divisor = pd.DataFrame([])
    idx_and_divisor["index"] = weights.index
    idx_and_divisor["mase_divisor"] = mase_divisor
    return idx_and_divisor

In [14]:
mase_val_divisor = calculate_mase_divisor("val").set_index("index")

Processing the val set:   0%|          | 0/9995 [00:00<?, ?it/s]

In [28]:
mase_train_divisor = calculate_mase_divisor("train").set_index("index")

Processing the train set:   0%|          | 0/90000 [00:00<?, ?it/s]

In [33]:
if WRITE_TO_FILE:
    mase_val_divisor.reset_index().to_feather(ROOT / "loss_functions/loss_func_val.feather")
    mase_train_divisor.reset_index().to_feather(ROOT / "loss_functions/loss_func_train.feather")